In [28]:
import pandas as pd
from shapely import wkt
from shapely.geometry import Polygon, MultiPolygon
import h3

In [10]:
df: pd.DataFrame = pd.read_parquet("../files/buildings.parquet.gz")

print(df.shape)
df.head()

(109066, 5)


,disaster_type,disaster,id,map_polygon,damage
0,flooding,hurricane-florence,MjU0NjYxOA.oDPPzoZ4qHe2KBGmwerMXsQbF9I,POLYGON ((-79.05237408501472 33.58644838741007...,no-damage
1,flooding,hurricane-florence,MjU0NjYxOA.oDPPzoZ4qHe2KBGmwerMXsQbF9I,POLYGON ((-79.05057793741786 33.58642007435035...,no-damage
2,flooding,hurricane-florence,MjU0NjYxOA.oDPPzoZ4qHe2KBGmwerMXsQbF9I,POLYGON ((-79.05032036154907 33.58601889360698...,no-damage
3,flooding,hurricane-florence,MjU0NjYxOA.oDPPzoZ4qHe2KBGmwerMXsQbF9I,POLYGON ((-79.04964714990703 33.58640540214724...,no-damage
4,flooding,hurricane-florence,MjU0NjYxOA.oDPPzoZ4qHe2KBGmwerMXsQbF9I,POLYGON ((-79.04905688085425 33.58639609760957...,no-damage


In [11]:
h3_df_8: pd.DataFrame = pd.read_csv("../files/merged_df_h3_cell_8.csv")

print(h3_df_8.shape)
h3_df_8.head()

(916266, 5)


,h3_cell_8,sum_population,avg_food,sum_gdp,gdp_per_capita
0,8826490093fffff,5.0,NaN,458617.030,91723.406000
1,8826490095fffff,2.0,NaN,NaN,NaN
2,8826490097fffff,15.0,NaN,108257.973,7217.198200
3,8826490099fffff,105.0,NaN,84240.760,802.292952
4,88264900a7fffff,3.0,NaN,67860.240,22620.080000


In [16]:
df["map_polygon_shape"] = df["map_polygon"].apply(wkt.loads)

In [56]:
df['h3_8_id'] = df['map_polygon_shape'].dropna().apply(lambda cell: h3.geo_to_h3(cell.centroid.y, cell.centroid.x, 8))

In [ ]:
h3_ids_with_neighbours: list[str] = list(df['h3_8_id'].dropna().apply(lambda cell: h3.k_ring(cell, 3)).explode().unique())
print(len(h3_ids_with_neighbours))

In [65]:
filtered_h3_8_ids: pd.DataFrame = h3_df_8[h3_df_8['h3_cell_8'].isin(h3_ids_with_neighbours)].reset_index(drop=True)
print(filtered_h3_8_ids.shape)
filtered_h3_8_ids.head()

(3409, 5)


,h3_cell_8,sum_population,avg_food,sum_gdp,gdp_per_capita
0,882ad01003fffff,9.0,NaN,14853.2790,1650.364333
1,882ad0100dfffff,26.0,NaN,4037.5083,155.288781
2,882ad0101bfffff,139.0,NaN,11148.0330,80.201676
3,882ad01021fffff,60.0,NaN,8387.8500,139.797500
4,882ad0102bfffff,27.0,NaN,NaN,NaN


In [68]:
filtered_h3_8_ids.to_parquet("../files/hurricanes_h3_8.parquet.gz", index=False, compression="gzip")